In [2]:
from dotenv import load_dotenv, find_dotenv
import os
_=load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
)

d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  warnings.warn(
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [4]:
# chatprompt template and str output parser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [5]:
chain= prompt | model | output_parser
chain.invoke({"soccer_player": "Lionel Messi"})

"Here's one:\n\n**Lionel Messi has a growth hormone deficiency**: Messi was diagnosed with a growth hormone deficiency when he was just 11 years old. This condition affected his growth and development, and he was only 4 feet 7 inches tall at the time. His family couldn't afford the treatment, which cost $1,500 per month, so FC Barcelona offered to pay for his treatment if he joined their youth academy. The rest, as they say, is history!\n\nIsn't that an amazing fact?"

Use of .bind() to add arguments to a Runnable in a LCEL Chain


In [6]:
chain= prompt | model.bind(stop=["Messi"]) | output_parser
chain.invoke({"soccer_player": "Lionel Messi"})

"Here's one:\n\nLionel "

Combining LCEL Chains


In [7]:
#Combining LCEL Chains
prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()
chain.invoke({"politician": "Barack Obama"})

'Here is a sentence about Barack Obama:\n\nBarack Obama was the 44th President of the United States, serving two terms from 2009 to 2017.'

Combined chain


In [8]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()
composed_chain.invoke({"politician": "Barack Obama"})

'What a great question!\n\nBarack Obama\'s presidency was indeed a significant milestone in American history, marking the first time an African American held the office of President. His presidency has had a profound impact on humanity, with many positive outcomes. Here are some examples:\n\n1. **Inspiration to People of Color**: Obama\'s election and presidency served as a beacon of hope and inspiration to people of color, particularly African Americans, who saw themselves reflected in the highest office in the land. It signaled that the doors of opportunity were open to them, and that they too could achieve greatness.\n2. **Racial Reconciliation**: Obama\'s presidency helped to bridge the racial divide in America, fostering greater understanding and empathy between people of different racial and ethnic backgrounds. His leadership promoted a sense of unity and inclusivity, which was essential for advancing social justice and equality.\n3. **Healthcare Reform**: The Affordable Care Act

Another example: a chain inside another chain


In [9]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)
chain1 = prompt1 | model | StrOutputParser()
chain2=(
    {"country": chain1,"language": itemgetter("language")}
    | prompt2 
    | model
    | StrOutputParser()
)
chain2.invoke({"politician": "Barack Obama", "language": "bangla"})

'Barack Obama, the 44th President of the United States, is an American by birth and nationality. He was born on August 4, 1961, in Honolulu, Hawaii, USA.\n\nSo, to answer your question, Barack Obama is from the United States of America, which is in the continent of উত্তর আমেরিকা (Uttor AmeriKa) or North America.'

LCEL chain at work in a typical RAG app


In [10]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [12]:
#hugging face embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

C:\Users\DELL\AppData\Local\Temp\ipykernel_5396\3866492197.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
d:\anaconda\envs\llmapp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [16]:
retriever= vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

d:\anaconda\envs\llmapp\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
rag_chain.invoke("What is Task Decomposition?")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
d:\anaconda\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
d:\anaconda\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at

'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. It can be done using various methods, including prompting a large language model (LLM) with simple instructions, using task-specific instructions, or with human inputs. This process helps agents plan ahead and understand what steps are involved in completing a complicated task.'

In [20]:
rag_chain.invoke("what is the difference between a task and a subtask?")

d:\anaconda\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
d:\anaconda\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'A task refers to a larger overall goal or objective, while a subtask is a smaller, more specific step that contributes to achieving the larger task. Task decomposition is the process of breaking down a complex task into smaller, manageable subtasks.'